In [ ]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '128g'})
import pandas as pd

In [2]:
# Load manifest and filter site of interest
manifest = hl.import_table("gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/BGE_NeuroGAP_Manifest.tsv", impute=True)
manifest = manifest.key_by('SUBJECT_ID')

# Clean SUBJECT_ID for matching and export (replace dots, spaces, or underscore with dash)
manifest = manifest.annotate(
    SUBJECT_ID_CLEAN = hl.str(manifest.SUBJECT_ID).replace(r'[._\s]', '-'),
    SEX = manifest.SEX,
    PHENOTYPE = manifest.ROLE,
    TERRA_WORKSPACE = manifest.TERRA_WORKSPACE
).key_by('SUBJECT_ID_CLEAN')

manifest_sub = manifest.filter(manifest.TERRA_WORKSPACE == "NeuroGAP-Psychosis_Uganda_Psychosis_BGE")
manifest_sub.show(5)  # Display first few rows

2025-07-28 23:00:06.619 Hail: INFO: Reading table to impute column types 1) / 1]
2025-07-28 23:00:09.012 Hail: INFO: Finished type imputation        (0 + 1) / 1]
  Loading field 'PROJECT' as type str (imputed)
  Loading field 'SUBJECT_ID' as type str (imputed)
  Loading field 'DATA_TYPE' as type str (imputed)
  Loading field 'PRIMARY_DISEASE' as type str (imputed)
  Loading field 'AGE_UNSPECIFIED' as type str (imputed)
  Loading field 'RACE_ETHNICITY' as type str (imputed)
  Loading field 'DISEASE_DESCRIPTION' as type str (imputed)
  Loading field 'VERSION' as type int32 (imputed)
  Loading field 'TERRA_WORKSPACE' as type str (imputed)
  Loading field 'INVESTIGATOR' as type str (imputed)
  Loading field 'EXTERNAL_PI' as type str (imputed)
  Loading field 'COHORT' as type str (imputed)
  Loading field 'ORSP_CG' as type str (imputed)
  Loading field 'CONSENT' as type str (imputed)
  Loading field 'SEQ_ID' as type str (imputed)
  Loading field 'COLLABORATOR_PARTICIPANT_ID' as type str (im

,,,,,,,,,,,,,,,,,,,,,,,,,,,
PROJECT,SUBJECT_ID,DATA_TYPE,PRIMARY_DISEASE,AGE_UNSPECIFIED,RACE_ETHNICITY,DISEASE_DESCRIPTION,VERSION,TERRA_WORKSPACE,INVESTIGATOR,EXTERNAL_PI,COHORT,ORSP_CG,CONSENT,SEQ_ID,COLLABORATOR_PARTICIPANT_ID,SM_ID,PDO,CAPTURE,SEX,CRAM,GVCF,ROLE,COVERAGE_MET,CHIMERA,CONTAMINATION,SUBJECT_ID_CLEAN,PHENOTYPE
str,str,str,str,str,str,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,float64,float64,str,str
"""RP-1691.PDO-29438""","""MAP1""","""BGE""","""Control""","""31""","""African""","""SCZ_BP""",1,"""NeuroGAP-Psychosis_Uganda_Psychosis_BGE""","""Koenen""","""Akena""","""NeuroGAP-Psychosis_Uganda""","""CG-5599""","""GRU""","""RP-1691.PDO-29438_MAP1""","""MAP28544999""","""SM-GN3VD""","""PDO-29438""","""broad_custom_exome_v1""","""Female""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/NeuroGAP-Psychosis_Uganda_Psychosis_BGE/RP-1691.PDO-29438/Exome/MAP1/v1/MAP1.cram""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/Uganda_gVCF/gvcfs/MAP1.v1.96223ab4-520b-42ba-a356-eb172208fab2.rb.g.vcf.gz""","""Control""","""2/14/23""",1.55e+00,5.00e-02,"""MAP1""","""Control"""
"""RP-1691.PDO-29438""","""MAP10""","""BGE""","""Control""","""18""","""African""","""SCZ_BP""",1,"""NeuroGAP-Psychosis_Uganda_Psychosis_BGE""","""Koenen""","""Akena""","""NeuroGAP-Psychosis_Uganda""","""CG-5599""","""GRU""","""RP-1691.PDO-29438_MAP10""","""MAP66243082""","""SM-GN3VM""","""PDO-29438""","""broad_custom_exome_v1""","""Male""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/NeuroGAP-Psychosis_Uganda_Psychosis_BGE/RP-1691.PDO-29438/Exome/MAP10/v1/MAP10.cram""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/Uganda_gVCF/gvcfs/MAP10.v1.7714ad9b-3cb9-45f1-a59a-272b8fdcddc9.rb.g.vcf.gz""","""Control""","""2/13/23""",1.73e+00,4.00e-02,"""MAP10""","""Control"""
"""RP-1691.PDO-29438""","""MAP100""","""BGE""","""Control""","""27""","""African""","""SCZ_BP""",1,"""NeuroGAP-Psychosis_Uganda_Psychosis_BGE""","""Koenen""","""Akena""","""NeuroGAP-Psychosis_Uganda""","""CG-5599""","""GRU""","""RP-1691.PDO-29438_MAP100""","""MAP65206400""","""SM-GN6PF""","""PDO-29438""","""broad_custom_exome_v1""","""Female""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/NeuroGAP-Psychosis_Uganda_Psychosis_BGE/RP-1691.PDO-29438/Exome/MAP100/v1/MAP100.cram""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/Uganda_gVCF/gvcfs/MAP100.v1.8d16a9e5-3275-4b01-acb0-7fe4a1c7593a.rb.g.vcf.gz""","""Control""","""2/14/23""",1.67e+00,4.20e-01,"""MAP100""","""Control"""
"""RP-1691.PDO-29438""","""MAP1000""","""BGE""","""Control""","""42""","""African""","""SCZ_BP""",1,"""NeuroGAP-Psychosis_Uganda_Psychosis_BGE""","""Koenen""","""Akena""","""NeuroGAP-Psychosis_Uganda""","""CG-5599""","""GRU""","""RP-1691.PDO-29438_MAP1000""","""MAP83154158""","""SM-I4D6S""","""PDO-29438""","""broad_custom_exome_v1""","""Female""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/NeuroGAP-Psychosis_Uganda_Psychosis_BGE/RP-1691.PDO-29438/Exome/MAP1000/v1/MAP1000.cram""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/Uganda_gVCF/gvcfs/MAP1000.v1.edfe93a4-4d71-40a7-8165-bacdb5ffd8f2.rb.g.vcf.gz""","""Control""","""2/6/23""",1.61e+00,8.00e-02,"""MAP1000""","""Control"""
"""RP-1691.PDO-31405""","""MAP10000""","""BGE""","""Psychotic Disorder NOS""","""50""","""African""","""SCZ_BP""",1,"""NeuroGAP-Psychosis_Uganda_Psychosis_BGE""","""Koenen""","""Akena""","""NeuroGAP-Psychosis_Uganda""","""CG-5599""","""GRU""","""RP-1691.PDO-31405_MAP10000""","""MAP99234450""","""SM-N9EUU""","""PDO-31405""","""broad_custom_exome_v1""","""Female""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/NeuroGAP-Psychosis_Uganda_Psychosis_BGE/RP-1691.PDO-31405/Exome/MAP10000/v1/MAP10000.cram""","""gs://fc-5058e99a-3e69-47a7-bc79-73aae6b19e95/Uganda_gVCF/gvcfs/MAP10000.v1.bd58c59a-a50e-407b-adff-4b604875335a.rb.g.vcf.gz""","""Case""","""6/27/23""",2.02e+00,6.00e-02,"""MAP10000""","""Case"""


In [3]:
# Remove duplicates and mismatches

# Read in duplicate IDs that were identified by `plink2 --king-cutoff 0.355`
# which was run to find the maximal independent set after removing duplicates/MZ twins
# these are the IDs we will remove first, as well as IDs identified to be "mismatches" between case and control status
# Note that this is in plink format, with FID and IID as the two columns, but these are the same ID in both columns

dups = hl.import_table("gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/dups_to_remove.txt", impute=True, delimiter='\t')
dup_ids = hl.set(dups.IID.collect())


2025-07-28 23:00:16.332 Hail: INFO: Reading table to impute column types
2025-07-28 23:00:17.067 Hail: INFO: Finished type imputation
  Loading field '#FID' as type str (imputed)
  Loading field 'IID' as type str (imputed)


In [4]:
# Read in duplicates with case/control status file
# Subset the duplicates table to only include instances where diagnosis does not match (same_primary_disease == False)
dup_info = hl.import_table("gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/neurogap_fullset.kin0_with_cohorts_roles_kinmin_354.csv", impute=True, delimiter=',')
mismatches = dup_info.filter(~dup_info.same_primary_disease)
mismatch_ids = hl.set(mismatches.ID1_SUBJECT_ID.collect() + mismatches.ID2_SUBJECT_ID.collect())

2025-07-28 23:01:31.874 Hail: INFO: Reading table to impute column types
2025-07-28 23:01:32.654 Hail: INFO: Finished type imputation
  Loading field 'ID1' as type str (imputed)
  Loading field 'ID1_SUBJECT_ID' as type str (imputed)
  Loading field 'ID1_COLLABORATOR_PARTICIPANT_ID' as type str (imputed)
  Loading field 'ID2' as type str (imputed)
  Loading field 'ID2_SUBJECT_ID' as type str (imputed)
  Loading field 'ID2_COLLABORATOR_PARTICIPANT_ID' as type str (imputed)
  Loading field 'N_SNP' as type int32 (imputed)
  Loading field 'HetHet' as type float64 (imputed)
  Loading field 'IBS0' as type int32 (imputed)
  Loading field 'HetConc' as type float64 (imputed)
  Loading field 'HomIBS0' as type float64 (imputed)
  Loading field 'Kinship' as type float64 (imputed)
  Loading field 'IBD1Seg' as type float64 (imputed)
  Loading field 'IBD2Seg' as type float64 (imputed)
  Loading field 'PropIBD' as type float64 (imputed)
  Loading field 'InfType' as type str (imputed)
  Loading field 'I

In [5]:
combined_remove_ids = dup_ids.union(mismatch_ids)
manifest_final = manifest_sub.filter(~combined_remove_ids.contains(manifest_sub.SUBJECT_ID))
final_sample_set = hl.set(manifest_final.SUBJECT_ID.collect())

2025-07-28 23:01:42.283 Hail: INFO: Ordering unsorted dataset with network shuffle


In [6]:
# Load and filter genotype data

input_path = f"gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/bgen/neurogap_final_merged_INFO0.8_rsIDs_chr*.bgen"

bgen = hl.import_bgen(input_path,entry_fields=['GT','GP'])

# Clean bgen sample IDs and rekey
bgen = bgen.annotate_cols(clean_s = hl.str(bgen.s).replace(r'[._\s]', '-'))
bgen = bgen.key_cols_by(s = bgen.clean_s)
bgen = bgen.filter_cols(final_sample_set.contains(bgen.s))

#checkpoint the subset matrixTable so it doesn't need to be re-computed
bgen = bgen.checkpoint('gs://neurogap-bge-imputed-regional/toni/temp/Uganda_prs_bgen_subset1.mt', overwrite=True)

2025-07-28 23:02:08.343 Hail: INFO: Number of BGEN files parsed: 22
2025-07-28 23:02:08.343 Hail: INFO: Number of samples in BGEN files: 38797
2025-07-28 23:02:08.343 Hail: INFO: Number of variants across all BGEN files: 32294864
2025-07-28 23:02:40.462 Hail: INFO: Number of BGEN files parsed: 22
2025-07-28 23:02:40.462 Hail: INFO: Number of samples in BGEN files: 38797
2025-07-28 23:02:40.462 Hail: INFO: Number of variants across all BGEN files: 32294864
2025-07-28 23:26:56.202 Hail: INFO: wrote matrix table with 32294864 rows and 10639 columns in 1087 partitions to gs://neurogap-bge-imputed-regional/toni/temp/Uganda_prs_bgen_subset1.mt


In [7]:
# Remove related individuals using KING kinship file
relatedness = hl.import_table("gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/neurogap_imputed_king_kinship_no_dup.txt", impute=True)
relatedness = relatedness.filter(final_sample_set.contains(relatedness.ID1) | final_sample_set.contains(relatedness.ID2))
related_samples = hl.maximal_independent_set(relatedness.ID1, relatedness.ID2, False)


2025-07-28 23:26:59.230 Hail: INFO: Reading table to impute column types
2025-07-28 23:26:59.793 Hail: INFO: Finished type imputation
  Loading field 'ID1' as type str (imputed)
  Loading field 'ID2' as type str (imputed)
  Loading field 'Kinship' as type float64 (imputed)
2025-07-28 23:27:08.700 Hail: INFO: wrote table with 882 rows in 1 partition to /tmp/uWtI3Dh0246LBwABSfVArO


In [8]:
# Filter to unrelated individuals
bgen = bgen.filter_cols(hl.is_defined(related_samples[bgen.col_key]), keep=False)


In [9]:
# Variant QC and MAF filter
bgen = hl.variant_qc(bgen)
bgen = bgen.filter_rows(bgen.variant_qc.AF[1] >= 0.05)


In [10]:
# Filter entries based on GP confidence (here threshold of 0.8 is used, anything lower is set to missing)
bgen = bgen.annotate_entries(
    GT = hl.or_missing(
        hl.max(bgen.GP) >= 0.8,
        bgen.GT
    )
)

In [11]:
# Filter variants to rsIDs in SNP list provided by PRS-CS
snplist = hl.import_table("gs://fc-728aff93-d8b3-439a-b002-91dd07198e77/NeuroGAP/rsids_1kg_hm3_afr.txt",no_header=True)
snplist = snplist.rename({'f0': 'rsid'}).key_by('rsid')
bgen = bgen.filter_rows(hl.is_defined(snplist.index(bgen.rsid)))


2025-07-28 23:27:09.851 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)


In [12]:
bgen = bgen.checkpoint('gs://neurogap-bge-imputed-regional/toni/temp/Uganda_prs_bgen_subset2.mt', overwrite=True)


2025-07-28 23:27:15.435 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-28 23:27:18.024 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-28 23:38:57.867 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-28 23:38:59.681 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-28 23:39:08.478 Hail: INFO: wrote table with 1225319 rows in 1 partition to /tmp/__iruid_1072-uEyGClCfKE0RfBqOLKHtey
2025-07-28 23:51:13.555 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-29 00:12:33.283 Hail: INFO: wrote matrix table with 1099581 rows and 9922 columns in 1087 partitions to gs://neurogap-bge-imputed-regional/toni/temp/Uganda_prs_bgen_subset2.mt


In [15]:
# Annotate phenotype, sex, and cleaned ID to plink format
bgen = bgen.annotate_cols(
    pheno = hl.case()
        .when(manifest[bgen.s].PHENOTYPE.lower() == 'case', 2)
        .when(manifest[bgen.s].PHENOTYPE.lower() == 'control', 1)
        .default(-9),
    is_female = hl.case()
        .when(manifest[bgen.s].SEX == 'Female', True)
        .when(manifest[bgen.s].SEX == 'Male', False)
        .default(hl.missing(hl.tbool))
)

In [16]:
# Export PLINK files with phenotype and sex

plink_path="gs://neurogap-bge-imputed-regional/toni/forPRSWiki/Uganda"       

hl.export_plink(
    bgen,
    output=plink_path,
    fam_id=bgen.s,
    ind_id=bgen.s,
    is_female=bgen.is_female
    pheno=bgen.pheno
    varid=bgen.rsid
)

2025-07-29 00:32:09.259 Hail: INFO: merging 1088 files totalling 2.5G... / 1087]
2025-07-29 00:32:52.821 Hail: INFO: while writing:
    gs://neurogap-bge-imputed-regional/toni/forPRSWiki/Uganda.bed
  merge time: 43.561s
2025-07-29 00:32:53.260 Hail: INFO: merging 1087 files totalling 43.3M...
2025-07-29 00:32:55.302 Hail: INFO: while writing:
    gs://neurogap-bge-imputed-regional/toni/forPRSWiki/Uganda.bim
  merge time: 2.042s
